In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Download traning data from open datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [4]:
# Load data
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}({X.dtype})")
    print(f"Shape of y: {y.shape}({y.dtype})")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])(torch.float32)
Shape of y: torch.Size([64])(torch.int64)


In [6]:
# Get cpu, gpu or mps device for training
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [37]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flattern = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flattern(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flattern): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [38]:
# Define loss, optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [39]:
# Define train func
def train(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn,
        optimizer: torch.optim.Optimizer,
    ):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [40]:
# Define test func
def test(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn,
    ):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

In [42]:
max_epochs = 50
target_accuracy = 0.9
for t in range(max_epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test_accuracy = test(test_dataloader, model, loss_fn)
    if test_accuracy >= target_accuracy:
        break

print("Done")

Epoch 1
-------------------------------
loss: 0.358606 [   64/60000]
loss: 0.487625 [ 6464/60000]
loss: 0.306165 [12864/60000]
loss: 0.536290 [19264/60000]
loss: 0.460815 [25664/60000]
loss: 0.477380 [32064/60000]
loss: 0.484509 [38464/60000]
loss: 0.660028 [44864/60000]
loss: 0.606249 [51264/60000]
loss: 0.437720 [57664/60000]
Test Error: 
 Accuracy: 82.7%, Avg loss: 0.487211 

Epoch 2
-------------------------------
loss: 0.355142 [   64/60000]
loss: 0.485776 [ 6464/60000]
loss: 0.304141 [12864/60000]
loss: 0.533464 [19264/60000]
loss: 0.457753 [25664/60000]
loss: 0.475264 [32064/60000]
loss: 0.482266 [38464/60000]
loss: 0.658817 [44864/60000]
loss: 0.604471 [51264/60000]
loss: 0.435199 [57664/60000]
Test Error: 
 Accuracy: 82.7%, Avg loss: 0.485573 

Epoch 3
-------------------------------
loss: 0.351765 [   64/60000]
loss: 0.483988 [ 6464/60000]
loss: 0.302188 [12864/60000]
loss: 0.530796 [19264/60000]
loss: 0.454770 [25664/60000]
loss: 0.473231 [32064/60000]
loss: 0.480093 [38464/

In [43]:
# Save model
save_path = "model/model.pth"
torch.save(model.state_dict(), save_path)
print("Saved PyTorch Model State to {save_path}")

Saved PyTorch Model State to {save_path}


In [44]:
# Load model
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(save_path))
print(model)

NeuralNetwork(
  (flattern): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [47]:
# Prediction
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
